In [ ]:
import torch
from transformers import TFLongformerModel, LongformerTokenizer
import pandas as pd
from sklearn import model_selection
import tensorflow as tf
import tensorflow_text as text
import numpy as np
from official.nlp import optimization

In [ ]:
# Load in dataset
data = pd.read_csv("../../data/sensitivity_data/sensitivity_dataset.csv")
data = data[["Filename","Date","Sensitivity","Document"]]

# Train / Test split
train_x, test_x, train_y, test_y = model_selection.train_test_split(data['Document'],data['Sensitivity'],test_size=0.2,random_state=5)

In [ ]:
model = TFLongformerModel.from_pretrained('allenai/longformer-base-4096')
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

In [ ]:
input_ids = tf.keras.Input(shape=(4096),dtype='int32')
attention_mask = tf.keras.Input(shape=(4096), dtype='int32')
global_attention_mask = None
outputs = model(input_ids, attention_mask=attention_mask,global_attention_mask=global_attention_mask)
pooled_output = outputs.pooler_output
dropout = tf.keras.layers.Dropout(0.1)(pooled_output)
dense = tf.keras.layers.Dense(1,activation="sigmoid")(dropout)
model = tf.keras.Model(inputs = [input_ids, attention_mask],outputs = dense)
model.summary()

In [ ]:
optimiser = tf.keras.optimizers.Adam()
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
metrics = tf.keras.metrics.BinaryAccuracy()

model.compile(optimizer=optimiser,loss='binary_crossentropy',metrics=metrics)

In [ ]:
inputs = tokenizer(list(train_x), return_tensors="tf",padding=True,truncation=True)
ids = inputs["input_ids"]
mask = inputs["attention_mask"]

In [46]:
model.fit(x=[ids,mask],y=train_y, epochs=5, batch_size=1, verbose=2)